In [1]:
from satellite_czml import satellite_czml
from satellite_czml import satellite
import pytz

from datetime import datetime, timedelta
import random

In [2]:
# # Simple test
# single = [['ISS (ZARYA)',
#   '1 25544U 98067A   21016.23305200  .00001366  00000-0  32598-4 0  9992',
#   '2 25544  51.6457  14.3113 0000235 231.0982 239.8264 15.49297436265049']]
# single_czml = satellite_czml(tle_list=single).get_czml()

In [3]:
import pandas as pd
target_time = datetime(2009, 2,10,16,56,0)

In [4]:
# Cosmos and Iridium Satellites before collision
df = pd.read_pickle('../satellites_go_boom/iridium_33_before_after.pkl')
df['CREATION_DATE'] = pd.to_datetime(df['CREATION_DATE'], format='%Y-%m-%dT%H:%M:%S')
tle1 = df[df['CREATION_DATE'] == (df[df['CREATION_DATE'] < target_time]['CREATION_DATE'].max())][['OBJECT_NAME','TLE_LINE1','TLE_LINE2']].values.tolist()[0]

df = pd.read_pickle('../satellites_go_boom/cosmos_2251_before_after.pkl')
df['CREATION_DATE'] = pd.to_datetime(df['CREATION_DATE'], format='%Y-%m-%dT%H:%M:%S')
tle2 = df[df['CREATION_DATE'] == (df[df['CREATION_DATE'] < target_time]['CREATION_DATE'].max())][['OBJECT_NAME','TLE_LINE1','TLE_LINE2']].values.tolist()[0]

tles = [tle1, tle2]

# This was attempting to create single CZML - gave up for now
# end_time = target_time + timedelta(minutes=30)
# start_time = target_time - timedelta(minutes=30)
# czml_obj = satellite_czml(tle_list=tles, start_time=start_time, end_time=end_time)
# for sat in czml_obj.satellites.values():
#     sat.end_time = target_time.replace(tzinfo=pytz.UTC)

end_time = target_time
start_time = end_time - timedelta(minutes=30)
czml_obj2 = satellite_czml(tle_list=tles, start_time=start_time, end_time=end_time)

single_czml2 = czml_obj2.get_czml()

In [4]:
# Debris after the collision
df = pd.read_pickle('../satellites_go_boom/cosmos_2251_deb_first_801.pkl')
# df['CREATION_DATE'] = pd.to_datetime(df['CREATION_DATE'], format='%Y-%m-%dT%H:%M:%S')
# df = df[df['CREATION_DATE'] <= '2009-02-28']
tles = df[['OBJECT_NAME','TLE_LINE1','TLE_LINE2']].values.tolist()

df = pd.read_pickle('../satellites_go_boom/iridium_33_deb_first_482.pkl')
# df['CREATION_DATE'] = pd.to_datetime(df['CREATION_DATE'], format='%Y-%m-%dT%H:%M:%S')
# df = df[df['CREATION_DATE'] <= '2009-02-28']
tles_b = df[['OBJECT_NAME','TLE_LINE1','TLE_LINE2']].values.tolist()

tles.extend(tles_b)
start_time = target_time
end_time = start_time + timedelta(minutes=30)
single_czml = satellite_czml(tle_list=tles, start_time=start_time, end_time=end_time,
                             use_default_image=False, show_label=False, show_path=False).get_czml()

In [5]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output


external_css = ['https://cesium.com/downloads/cesiumjs/releases/1.76/Build/Cesium/Widgets/widgets.css']

external_scripts = [{'src':'https://cesium.com/downloads/cesiumjs/releases/1.76/Build/Cesium/Cesium.js'}]

app = dash.Dash(__name__,
                title='satellite_czml',
                external_scripts=external_scripts,
                external_stylesheets=external_css)

app.layout = html.Div(children=[
    dcc.Dropdown(id='choose_czml_dd',
                 options=[
                     {'label': 'Single CZML', 'value': 'single_czml'}
                 ],
                 value='single_czml'
                ),
    html.Div(id='cesiumContainer'),
    html.Div(id='czml', style={'display': 'none'}),
    html.Div(id='apikey', style={'display': 'none'}, children="ZXlKaGJHY2lPaUpJVXpJMU5pSXNJblI1Y0NJNklrcFhWQ0o5LmV5SnFkR2tpT2lJd05tSXdORGN5WkMwNE5tUTBMVFExTnpRdFltVTNOeTAxWVRabFpUVTRNRFUzWkRVaUxDSnBaQ0k2TkRBeE5ESXNJbWxoZENJNk1UWXdPRE0xTkRZNE9IMC5uT1pBQ291ay0tZnhQX2V1cXRnRmt3d05TMi02NEJaODFBTWVNbzlwZ1lj")
])

@app.callback(
    Output('czml', 'children'),
    Input('choose_czml_dd', 'value'))
def update_satellite_filter(v):
    d = {'single_czml':single_czml
        }
    return d[v]

app.clientside_callback('''
function(id, czml, apikey) {
    // Create the Cesium Viewer
    if (!window.viewer) {
        Cesium.Ion.defaultAccessToken = atob(apikey);
        window.viewer = new Cesium.Viewer(id,{
            shouldAnimate: true,
        });
        window.viewer.scene.globe.enableLighting = true;
    }

    // Update the Cesium Viewer
    if (czml) {
        window.viewer.dataSources.removeAll();
        czmlJson = JSON.parse(czml);
        window.viewer.dataSources.add(
            Cesium.CzmlDataSource.load(czmlJson)
        );
    }

    return true;
}''',
    Output('cesiumContainer', 'data-done'),
    Input('cesiumContainer', 'id'),
    Input('czml', 'children'),
    Input('apikey', 'children')
)

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
